In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import base64
import numpy as np
import requests
import gzip
from os import listdir
from os.path import isfile, join
from random import shuffle,choice
from time import sleep
import random

In [2]:
def arruma(series):
    return pd.to_numeric(series, errors='coerse', downcast ='integer')

In [3]:
data = pd.read_pickle('data.pickle')
data['MovieYear'] = arruma(data['MovieYear'])
data['MovieImdbRating'] = arruma(data['MovieImdbRating'])
data = data[data['MovieYear'] > 2010]
data = data[data['MovieImdbRating'] > 6]
data.sort_values(['MovieYear','MovieImdbRating'], ascending=False, inplace=True)

print(data.shape)
data.head() 

(35195, 59)


,IDMovie,IDMovieImdb,IDSubMovieFile,IDSubtitle,IDSubtitleFile,ISO639,InfoFormat,InfoOther,InfoReleaseGroup,LanguageName,MatchedBy,MovieByteSize,MovieFPS,MovieHash,MovieImdbRating,MovieKind,MovieName,MovieNameEng,MovieReleaseName,MovieTimeMS,MovieYear,QueryCached,QueryNumber,QueryParameters,Score,SeriesEpisode,SeriesIMDBParent,SeriesSeason,SubActualCD,SubAddDate,SubAuthorComment,SubAutoTranslation,SubBad,SubComments,SubDownloadLink,SubDownloadsCnt,SubEncoding,SubFeatured,SubFileName,SubForeignPartsOnly,SubFormat,SubFromTrusted,SubHD,SubHash,SubHearingImpaired,SubLanguageID,SubLastTS,SubRating,SubSize,SubSumCD,SubSumVotes,SubTSGroup,SubTSGroupHash,SubTranslator,SubtitlesLink,UserID,UserNickName,UserRank,ZipDownloadLink
0,724769,8426840,0,7683035,1956257965,en,HDTV,Proper,224Kbps) - 900MB - ESub,English,imdbid,0,23.976,0,9.1,movie,Yatra,,Yatra (2019) Telugu Proper HDRip - 720p - x265...,0,2019.0,1.0,0,"{'imdbid': '8426840', 'sublanguageid': 'eng'}",21.00560,0,0,0,1,2019-03-11 08:59:50,,0,0,0,http://dl.opensubtitles.org/en/download/src-ap...,560,UTF-8,0,Yatra (2019) Telugu Proper HDRip - 720p - x265...,0,srt,0,1,fd61e9c4c58ae4712e16dc3842789bc2,0,eng,02:02:54,0.0,132873,1,0,5,44eb75df9d3f9afd1b2d1c13773a29e1,,http://www.opensubtitles.org/en/subtitles/7683...,6997096,NARACHI,platinum member,http://dl.opensubtitles.org/en/download/src-ap...
1,724769,8426840,0,7685845,1956260747,en,WEBRip,,Amazon,English,imdbid,0,23.976,0,9.1,movie,Yatra,,Yatra.2019.Tamil.Proper.HQ.HDRip.x264.700MB-ESubs,0,2019.0,1.0,0,"{'imdbid': '8426840', 'sublanguageid': 'eng'}",10.00076,0,0,0,1,2019-03-14 07:40:20,,0,0,0,http://dl.opensubtitles.org/en/download/src-ap...,76,UTF-8,0,Yatra.(Telugu).WEBRip.Amazon.en-gb.srt,0,srt,0,0,f32b780e85756b8a51e4993cc7e522c4,0,eng,02:03:00,0.0,126936,1,0,6,063c7afb0aaea40c3cce76fb0cf38e84,,http://www.opensubtitles.org/en/subtitles/7685...,0,,,http://dl.opensubtitles.org/en/download/src-ap...
2,724769,8426840,0,7685844,1956260746,en,WEBRip,,Amazon,English,imdbid,0,23.976,0,9.1,movie,Yatra,,Yatra.2019.Tamil.Proper.HQ.HDRip.x264.700MB-ESubs,0,2019.0,1.0,0,"{'imdbid': '8426840', 'sublanguageid': 'eng'}",10.00036,0,0,0,1,2019-03-14 07:39:49,,0,0,0,http://dl.opensubtitles.org/en/download/src-ap...,36,UTF-8,0,Yatra.(Telugu).WEBRip.Amazon.en-gb.srt,0,srt,0,0,9027b3b52dec8c42b114c302af22fdca,1,eng,02:03:00,0.0,132384,1,0,6,063c7afb0aaea40c3cce76fb0cf38e84,,http://www.opensubtitles.org/en/subtitles/7685...,0,,,http://dl.opensubtitles.org/en/download/src-ap...
0,710483,8291224,0,7674121,1956249062,en,HDTV,,ESubs - KGF,English,imdbid,0,23.976,0,9.0,movie,Uri: The Surgical Strike,,URI The Surgical Strike (2019) Hindi - 1080p -...,0,2019.0,1.0,0,"{'imdbid': '8291224', 'sublanguageid': 'eng'}",21.42493,0,0,0,1,2019-03-02 12:27:28,,0,0,0,http://dl.opensubtitles.org/en/download/src-ap...,42493,UTF-8,0,URI The Surgical Strike (2019) Hindi - 1080p -...,0,srt,0,1,1e6517e8c8b7c01d03e94dd2f73df4ae,0,eng,02:17:35,0.0,97905,1,0,10,3d66cf8c9979d41aa40e7f2e99b0205c,,http://www.opensubtitles.org/en/subtitles/7674...,6914205,_Jio_,platinum member,http://dl.opensubtitles.org/en/download/src-ap...
1,710483,8291224,0,7677699,1956252638,en,HDTV,,1.4GB,English,imdbid,0,23.976,0,9.0,movie,Uri: The Surgical Strike,,URI The Surgical Strike (2019) Hindi HDRip - 7...,0,2019.0,1.0,0,"{'imdbid': '8291224', 'sublanguageid': 'eng'}",16.01360,0,0,0,1,2019-03-06 09:15:21,,0,0,0,http://dl.opensubtitles.org/en/download/src-ap...,1360,UTF-8,0,URI The Surgical Strike (2019) Hindi HDRip - 7...,0,srt,0,1,de513a93b2099590ba0179b1764c2923,0,eng,02:17:40,0.0,96164,1,0,14,daa0fb38db18fd49bfbb7dfae1aac6ca,,http://www.opensubtitles.org/en/subtitles/7677...,5010135,jagy007,bronze member,http://dl.opensubtitles.org/en/download/src-ap...


In [147]:
list(data.ZipDownloadLink)[:6]

['http://dl.opensubtitles.org/en/download/src-api/vrf-f55d0bb9/sid-9scLyR-FuNbaPboOSGd9LNAuuic/sub/7683035',
 'http://dl.opensubtitles.org/en/download/src-api/vrf-f57f0bc4/sid-9scLyR-FuNbaPboOSGd9LNAuuic/sub/7685845',
 'http://dl.opensubtitles.org/en/download/src-api/vrf-f57e0bc3/sid-9scLyR-FuNbaPboOSGd9LNAuuic/sub/7685844',
 'http://dl.opensubtitles.org/en/download/src-api/vrf-f5590bb5/sid-GmEsDHz3T4yXcT4KjobC5dmhHt9/sub/7674121',
 'http://dl.opensubtitles.org/en/download/src-api/vrf-f58a0bcc/sid-GmEsDHz3T4yXcT4KjobC5dmhHt9/sub/7677699',
 'http://dl.opensubtitles.org/en/download/src-api/vrf-f55d0bb6/sid-GmEsDHz3T4yXcT4KjobC5dmhHt9/sub/7675121']

In [148]:
from multiprocessing import Pool

mypath='subs/'
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]


user_agent_list = [
   #Chrome
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    #Firefox
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)'
]



lista = list(data.SubDownloadLink)
shuffle(lista)

#proxies = list(get_proxies())
def reseta():
    global user_agent_list    
    global proxies
    user_agent = choice(user_agent_list)
    header = {'User-Agent': user_agent}
    proxy = {'http' : choice(proxies),'https' : choice(proxies)}
    return header,proxy


def download_file(url):
    #header,proxy = reseta()
    name = url.split('/')[-1]
    name = str(name.split('.')[0])+ '.txt'
    if name not in files:
        try:
            fullname = mypath  + name
            raw_data = requests.get(url) #, proxies=proxy, headers=header)
            with open(fullname, 'wb+') as file:
                content = gzip.decompress(raw_data.content)
                file.write(content)
                print('O',end='', flush=True)
                sleep(random.random()+.5)
        except Exception as e:
            print('E',end='', flush=True)
    else:
        print('_',end='', flush=True)

with Pool(4) as p:
    p.map(download_file, lista)

OSError: [Errno 22] Invalid argument

In [149]:
data.SubtitlesLink[:5]

0    http://www.opensubtitles.org/en/subtitles/7683...
1    http://www.opensubtitles.org/en/subtitles/7685...
2    http://www.opensubtitles.org/en/subtitles/7685...
0    http://www.opensubtitles.org/en/subtitles/7674...
1    http://www.opensubtitles.org/en/subtitles/7677...
Name: SubtitlesLink, dtype: object

In [150]:
data.ZipDownloadLink.head()

0    http://dl.opensubtitles.org/en/download/src-ap...
1    http://dl.opensubtitles.org/en/download/src-ap...
2    http://dl.opensubtitles.org/en/download/src-ap...
0    http://dl.opensubtitles.org/en/download/src-ap...
1    http://dl.opensubtitles.org/en/download/src-ap...
Name: ZipDownloadLink, dtype: object

In [151]:
from torrequest import TorRequest
tr=TorRequest(password='juju')


OSError: [Errno 22] Invalid argument

In [47]:
import requests
response= requests.get('http://ipecho.net/plain')
print("My Original IP Address:",response.text)

My Original IP Address: 179.218.85.20


In [48]:
def reset_ip():
    tr=TorRequest(password='juju')
    tr.reset_identity() #Reset Tor
    response= tr.get('http://ipecho.net/plain')
    print("New Ip Address",response.text)
reset_ip()

New Ip Address 185.220.101.52


In [24]:
import requests
from lxml.html import fromstring

def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:10]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies
proxies = get_proxies()
proxies

{'125.27.10.226:60657', '5.141.244.148:34185', '87.103.196.39:50629'}

In [50]:
#  https://www.scrapehero.com/make-anonymous-requests-using-tor-python/

In [51]:
reset_ip()

New Ip Address 163.172.66.247


In [ ]:
mypath='subs/'

files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

lista = list(data.SubDownloadLink)
shuffle(lista)
# print(lista[:20])

def download_file(url):
    name = url.split('/')[-1]
    name = str(name.split('.')[0])+ '.txt'
    if name not in files:
        try:
            fullname = mypath  + name
            raw_data = requests.get(url)
            with open(fullname, 'wb+') as file:
                content = gzip.decompress(raw_data.content)
                file.write(content)
                #print('.',end=' ', flush=True)
                print(name,len(content))
                #sleep(random.random()+.5)
        except:
            print(name, 'erro')
            return 'erro'
    else:
        print(name, 'já tinha')
i = 0
for url in lista:
    if download_file(url) == 'erro':
        #reset_ip()
        i += 1
        sleep(10)

1954653381.txt já tinha
1953061102.txt já tinha
1955420173.txt 132486
1953520075.txt já tinha
1954778049.txt já tinha
1955880096.txt 84484
1956019588.txt 73704
1954225019.txt já tinha
1955977027.txt já tinha
1954859997.txt já tinha
1956277841.txt já tinha
1954669993.txt já tinha
1955038422.txt já tinha
1952922152.txt já tinha
1954779939.txt já tinha
1955724115.txt 70991
1954852658.txt já tinha
1954837070.txt 85915
1955486308.txt já tinha
1954799189.txt 133444
1956146239.txt já tinha
1953569620.txt já tinha
1953536023.txt já tinha
1955439296.txt 53546
1955364243.txt já tinha
1953383160.txt já tinha
1955476439.txt 60257
1955349625.txt já tinha
1955436212.txt já tinha
1954918385.txt 70642
1955953607.txt já tinha
1955323201.txt já tinha
1955401888.txt já tinha
1955578118.txt 76361
1954799199.txt já tinha
1955530449.txt 88627
1954124152.txt já tinha
1955925797.txt já tinha
1953391921.txt 98648
1952886348.txt já tinha
1955391660.txt já tinha
1955108651.txt já tinha
1952863014.txt 100857
1954

1956197382.txt 44865
1955304265.txt já tinha
1953200911.txt já tinha
1953905043.txt já tinha
1955703842.txt já tinha
1955756032.txt 52926
1953493124.txt já tinha
1952939837.txt já tinha
1955399878.txt 90881
1955902611.txt já tinha
1956090870.txt já tinha
1954230549.txt já tinha
1955106411.txt já tinha
1955761308.txt já tinha
1956090869.txt 73747
1953786854.txt 168307
1955680474.txt já tinha
1952943997.txt já tinha
1955008894.txt já tinha
1955196495.txt já tinha
1953064887.txt 120853
1954792829.txt já tinha
1956176947.txt já tinha
1954753199.txt já tinha
1953739209.txt já tinha
1954881182.txt já tinha
1955504891.txt já tinha
1954374975.txt já tinha
1954481619.txt já tinha
1956144207.txt já tinha
1953604814.txt já tinha
1955260760.txt já tinha
1955795753.txt 51630
1955693895.txt já tinha
1952911862.txt já tinha
1955145865.txt já tinha
1955498665.txt já tinha
1954464646.txt 69677
1953426424.txt já tinha
1953090105.txt já tinha
1955289530.txt 95278
1955942241.txt 46642
1954157065.txt já ti

1955498760.txt erro
1955507582.txt já tinha
1953062159.txt já tinha
1955044156.txt já tinha
1953213578.txt já tinha
1953334534.txt já tinha
1956225117.txt já tinha
1954484753.txt erro
1955985248.txt erro
1955963676.txt já tinha
1953616853.txt já tinha
1956165542.txt erro
1953066051.txt já tinha
1953025240.txt já tinha
1955260764.txt já tinha
1955778769.txt erro
1954190517.txt já tinha
1955639380.txt erro
1954955267.txt já tinha
1955528361.txt já tinha
1954579511.txt erro
1953027783.txt erro
1953153591.txt erro
1954597577.txt erro
1955801337.txt erro
1955847095.txt erro
1956247816.txt já tinha
1954660762.txt já tinha
1956092465.txt erro
1954328986.txt já tinha
1955007443.txt erro
1955809467.txt erro
1953230863.txt já tinha
1954995552.txt já tinha
1954763742.txt erro
1952983874.txt já tinha
1953521417.txt erro
1955477191.txt já tinha
1953705419.txt já tinha
1953092038.txt já tinha
1956069249.txt já tinha
1955956619.txt erro
1954808815.txt já tinha
1954928545.txt erro
1953819057.txt já ti

In [4]:
import xmlrpc.client as xrpc
import base64
import gzip
import json


mypath='subs/'


server = xrpc.ServerProxy("http://api.opensubtitles.org/xml-rpc")

def baixa(subtitle_id):
    result = server.DownloadSubtitles(token, subtitle_id)
    for d in result['data']:
        filename = str(d['idsubtitlefile'])+".txt"
        if filename not in files:
            #print(filename)
            with open(mypath + filename, 'wb+') as file:
                content = gzip.decompress(base64.b64decode(d['data']))
                print('.',end='',flush=True)
                file.write(content) #.decode()))
#baixa([1956249062,1952281521,1952503920])

credentials_list = [server.LogIn("randomwalker", "sub1machine", "en", "MachineTitle"),
                   server.LogIn("omerakgul58", "omeromer", "en", "2016experimentingwithnlp"),
                   server.LogIn("mrkopp", "mymoviez00", "en", "SubtitleClassification"),
                   server.LogIn("mrkopp2", "mymoviez00", "en", "SubtitleClassification"),
                    server.LogIn("mrkopp4", "mymoviez00", "en", "SubtitleClassification"),
                   server.LogIn("mrkopp5", "mymoviez00", "en", "SubtitleClassification"),
                   server.LogIn("mrkopp7", "mymoviez00", "en", "SubtitleClassification"),
                   server.LogIn("mrkopp8", "mymoviez00", "en", "SubtitleClassification"),
                   server.LogIn("mrkopp9", "mymoviez00", "en", "SubtitleClassification")]
shuffle(credentials_list)
#credentials_list = [server.LogIn("mrkopp9", "mymoviez00", "en", "SubtitleClassification")]

for credentials in credentials_list:
    #credentials['data']['IsVIP'] = 1
    #credentials['data']['DownloadCnt'] = 1
    print(credentials)
    token = credentials.get("token")
    info = server.ServerInfo()
    #print(info)
    print(info['download_limits']['client_24h_download_count'])
    for f in range(20):
        try:
            h=20
            files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
            ja_tem = [x[:-4] for x in files]
            falta = [x for x in list(data.IDSubtitleFile) if x not in ja_tem]
            shuffle(falta)
            print()
            print(falta[f*h:f*h+h])
            baixa(falta[f*h:f*h+h])
            sleep(1)
        except Exception as e:
            print(e)
            break

{'token': 'CEFc5p-DHJ5dneQp0bQ-OIolN67', 'status': '200 OK', 'data': {'IDUser': '4426264', 'UserNickName': 'omerakgul58', 'UserRank': 'app developer', 'UploadCnt': '0', 'DownloadCnt': '5495', 'UserWebLanguage': 'en', 'IsVIP': 0}, 'seconds': 0.274}
0

['1956266347', '1956247946', '1955176125', '1954351621', '1955954129', '1956241227', '1955845609', '1953294968', '1954399522', '1956159890', '1956261594', '1954091875', '1954503451', '1955848742', '1955584223', '1955643494', '1955556293', '1955692954', '1953647940', '1955780811']
....................
['1954841867', '1954948751', '1955933038', '1955837583', '1954354274', '1956035486', '1955789788', '1955790656', '1956281291', '1952907719', '1953269348', '1956287580', '1954594512', '1956228883', '1955398707', '1955425987', '1955568389', '1955768707', '1955495347', '1954707854']
....................
['1956178670', '1956147382', '1955727737', '1954674783', '1953166085', '1954654782', '1955242234', '1953309377', '1955617085', '1955954120', '195

....................
['1956158026', '1955842704', '1953180191', '1953274955', '1955631555', '1954760060', '1955655002', '1953450591', '1955805034', '1953606200', '1955008761', '1955882190', '1955949856', '1954969933', '1954047313', '1955612014', '1955939883', '1953194679', '1954840740', '1956011239']
....................
['1955530171', '1955622706', '1955734541', '1954896170', '1953631245', '1956204602', '1955816431', '1953610254', '1956268627', '1954511218', '1956251506', '1954763791', '1952889284', '1953673567', '1954507191', '1956095337', '1954747338', '1953389145', '1954608536', '1955582715']
....................
['1955201714', '1955453573', '1955560058', '1955677201', '1956249907', '1956215735', '1955655198', '1955586023', '1954904164', '1953330863', '1955757374', '1955588014', '1953273182', '1955953575', '1955951640', '1953816596', '1955349299', '1955455231', '1953279454', '1955286125']
....................
['1955791634', '1955377335', '1955371703', '1953205405', '1955513629', '1

....................
['1955276154', '1953118391', '1955786007', '1955055074', '1955602446', '1954091685', '1955233502', '1956119465', '1955104078', '1955870900', '1955642564', '1955012262', '1955807086', '1954315678', '1954449991', '1953611286', '1955372818', '1955005353', '1955646854', '1955839615']
....................
['1956071591', '1955845513', '1956230615', '1953175707', '1955153680', '1952966926', '1955605160', '1953398211', '1955137435', '1954311001', '1954706395', '1955391241', '1953089621', '1955407784', '1955772150', '1955644607', '1954815584', '1954427438', '1955932930', '1953338562']
....................
['1954913488', '1955667412', '1956268395', '1955419926', '1956246019', '1954055293', '1955594051', '1954550073', '1955283003', '1955627559', '1955240662', '1955112936', '1954168669', '1955788908', '1952893019', '1956078448', '1952893579', '1953079521', '1955444229', '1956205738']
....................
['1955888283', '1956200743', '1955851763', '1955351915', '1954499037', '1

....................
['1955573186', '1955819189', '1954917522', '1955676838', '1955976581', '1954622816', '1952985634', '1955414247', '1955797767', '1955397211', '1955838653', '1954826555', '1955155926', '1955759347', '1955812380', '1955606953', '1953280219', '1956247225', '1956141561', '1954423697']
....................
['1953523765', '1953630094', '1955872925', '1953406692', '1953619637', '1955558207', '1956213428', '1955819610', '1956217182', '1953712159', '1953097709', '1955968367', '1955563031', '1956088215', '1956216646', '1955543043', '1953246303', '1953025208', '1953530630', '1954128238']
....................
['1955165195', '1956197216', '1954099062', '1953251051', '1952840400', '1954763749', '1953317412', '1956181890', '1956280909', '1955635340', '1953382667', '1954031268', '1955909486', '1954687471', '1956276685', '1955880331', '1955029297', '1956228882', '1956238103', '1953038882']
....................
['1955529490', '1955461048', '1953119661', '1956009362', '1955107774', '1

....................
['1953673370', '1955763819', '1955759877', '1955683216', '1952971670', '1953168473', '1956138245', '1955268318', '1956144334', '1956166721', '1954114499', '1954873898', '1955448996', '1954869227', '1955263208', '1955324115', '1955867203', '1955902778', '1955328180', '1953510428']
....................
['1956116742', '1956247679', '1954363536', '1954392950', '1953683771', '1955630100', '1953077255', '1954894269', '1953162415', '1955573799', '1955956991', '1956285427', '1954838456', '1955744626', '1954299985', '1956056214', '1955999101', '1953170901', '1955810119', '1955612532']
....................
['1956152757', '1954569013', '1955129123', '1953519577', '1952972666', '1955656092', '1955877936', '1956119473', '1956137926', '1955011852', '1953038625', '1956211524', '1956050718', '1955196948', '1956263644', '1953650614', '1954982189', '1955923627', '1955862343', '1955025453']
'bool' object is not iterable


In [6]:
def get_info(id_imdb):
    data = server.SearchSubtitles(token, [ {'imdbid': int(id_imdb[2:]), 'sublanguageid': 'eng'}])
    try:
        return pd.DataFrame.from_dict(data['data'])
    except:
        return False
    
get_info('tt5773402')

,IDMovie,IDMovieImdb,IDSubMovieFile,IDSubtitle,IDSubtitleFile,ISO639,InfoFormat,InfoOther,InfoReleaseGroup,LanguageName,MatchedBy,MovieByteSize,MovieFPS,MovieHash,MovieImdbRating,MovieKind,MovieName,MovieNameEng,MovieReleaseName,MovieTimeMS,MovieYear,QueryCached,QueryNumber,QueryParameters,Score,SeriesEpisode,SeriesIMDBParent,SeriesSeason,SubActualCD,SubAddDate,SubAuthorComment,SubAutoTranslation,SubBad,SubComments,SubDownloadLink,SubDownloadsCnt,SubEncoding,SubFeatured,SubFileName,SubForeignPartsOnly,SubFormat,SubFromTrusted,SubHD,SubHash,SubHearingImpaired,SubLanguageID,SubLastTS,SubRating,SubSize,SubSumCD,SubSumVotes,SubTSGroup,SubTSGroupHash,SubTranslator,SubtitlesLink,UserID,UserNickName,UserRank,ZipDownloadLink
0,662495,5773402,0,7506600,1956083316,en,,,,English,imdbid,0,25.000,0,9.3,movie,Dominion,,Dominion.English,0,2018,1,0,"{'imdbid': '5773402', 'sublanguageid': 'eng'}",10.00471,0,0,0,2,2018-10-11 16:25:34,,0,0,0,http://dl.opensubtitles.org/en/download/src-ap...,471,UTF-8,0,Dominion.English_w_quotes.srt,0,srt,0,0,767cc4c5fafe24cadf536eb39806d93b,0,eng,01:56:11,0.0,99705,2,0,1,89ba8a0cdc6a0801c22146f90caf8e05,,http://www.opensubtitles.org/en/subtitles/7506...,0,,,http://dl.opensubtitles.org/en/download/src-ap...
1,662495,5773402,0,7506600,1956083315,en,,,,English,imdbid,0,25.000,0,9.3,movie,Dominion,,Dominion.English,0,2018,1,0,"{'imdbid': '5773402', 'sublanguageid': 'eng'}",10.00471,0,0,0,1,2018-10-11 16:25:34,,0,0,0,http://dl.opensubtitles.org/en/download/src-ap...,471,UTF-8,0,Dominion.English.srt,0,srt,0,0,83e5d2c06ec21961fe9959cacaf53f9b,0,eng,01:55:29,0.0,95466,2,0,1,607b76559d5269bbd591e0606bfeb70b,,http://www.opensubtitles.org/en/subtitles/7506...,0,,,http://dl.opensubtitles.org/en/download/src-ap...


['1954326206', '1953282107', '1954647750', '1953110257', '1952863405', '1955230823', '1955473856', '1955719289', '1956096298', '1954164863', '1954575855', '1955354691', '1955401861', '1955263003', '1954097275', '1954961015', '1955235316', '1952843897', '1954956785', '1953080921']
'bool' object is not iterable


In [33]:

ja_tem

['123093',
 '1951858260',
 '1951858261',
 '1951858262',
 '1952148151',
 '1952262649',
 '1952281521',
 '1952283527',
 '1952291510',
 '1952301334',
 '1952308165',
 '1952318129',
 '1952319371',
 '1952322030',
 '1952325866',
 '1952327862',
 '1952328255',
 '1952331485',
 '1952338806',
 '1952340265',
 '1952340874',
 '1952342263',
 '1952347931',
 '1952349268',
 '1952351485',
 '1952353662',
 '1952356173',
 '1952357939',
 '1952360552',
 '1952361954',
 '1952362332',
 '1952364864',
 '1952365680',
 '1952372913',
 '1952377185',
 '1952393071',
 '1952394252',
 '1952406697',
 '1952409311',
 '1952410309',
 '1952412911',
 '1952425472',
 '1952433764',
 '1952447094',
 '1952447776',
 '1952454896',
 '1952460701',
 '1952467951',
 '1952472586',
 '1952472782',
 '1952476779',
 '1952481313',
 '1952488578',
 '1952490880',
 '1952498901',
 '1952500476',
 '1952503920',
 '1952504272',
 '1952512756',
 '1952518980',
 '1952530160',
 '1952537365',
 '1952538065',
 '1952538857',
 '1952541538',
 '1952546449',
 '1952560384',

In [125]:

falta

['1953075890',
 '1954197100',
 '1954668873',
 '1954125830',
 '1954221587',
 '1953381558',
 '1955308911',
 '1954042511',
 '1954157066',
 '1953281172',
 '1953030335',
 '1955374037',
 '1954611275',
 '1955795315',
 '1953296150',
 '1953109324',
 '1953033818',
 '1955631333',
 '1955011651',
 '1953449729',
 '1953091884',
 '1954415643',
 '1955889479',
 '1956026307',
 '1956118889',
 '1955907279',
 '1954763749',
 '1953608355',
 '1955898285',
 '1953046449',
 '1954391198',
 '1953706913',
 '1955371491',
 '1956068249',
 '1955140706',
 '1955033075',
 '1955893290',
 '1954620397',
 '1955120774',
 '1954765990',
 '1955036970',
 '1954241076',
 '1956024612',
 '1954864969',
 '1955082141',
 '1952948853',
 '1953281873',
 '1954443937',
 '1953109761',
 '1956222009',
 '1955340436',
 '1953063596',
 '1955352130',
 '1952962516',
 '1953159754',
 '1954894954',
 '1953024651',
 '1955071676',
 '1954967359',
 '1954949818',
 '1955409598',
 '1956080722',
 '1955024450',
 '1953811771',
 '1953132373',
 '1952966960',
 '19544627

In [12]:
baixa([0])

TypeError: baixa() missing 1 required positional argument: 'server'

In [138]:
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
ja_tem = [x[:-4] for x in files]
falta = [x for x in list(data.IDSubtitleFile) if x not in ja_tem]
len(ja_tem)

20526

In [139]:
len(falta)

10210

In [38]:
a = server.LogIn("mrkopp9", "mymoviez00", "en", "SubtitleClassification")

In [39]:
print(a)

{'token': 'orA5H,Rc4I2m08xdM9-YbNIbRd2', 'status': '200 OK', 'data': {'IDUser': '7067393', 'UserNickName': 'mrkopp9', 'UserRank': 'sub leecher', 'UploadCnt': '0', 'DownloadCnt': '212', 'UserWebLanguage': 'en', 'IsVIP': 0}, 'seconds': 0.064}


In [42]:
a['data']['IsVIP'] = 1
a['data']['DownloadCnt'] = 1

In [43]:
a

{'token': 'orA5H,Rc4I2m08xdM9-YbNIbRd2',
 'status': '200 OK',
 'data': {'IDUser': '7067393',
  'UserNickName': 'mrkopp9',
  'UserRank': 'sub leecher',
  'UploadCnt': '0',
  'DownloadCnt': '212',
  'UserWebLanguage': 'en',
  'IsVIP': 1},
 'seconds': 0.064}

In [71]:
ja_tem

['123093',
 '136803',
 '1951858260',
 '1951858261',
 '1951858262',
 '1952148151',
 '1952262649',
 '1952281521',
 '1952283527',
 '1952291510',
 '1952301334',
 '1952308165',
 '1952318129',
 '1952319371',
 '1952322030',
 '1952325866',
 '1952327862',
 '1952328255',
 '1952331485',
 '1952338806',
 '1952340265',
 '1952340874',
 '1952342263',
 '1952347931',
 '1952349268',
 '1952351485',
 '1952353662',
 '1952356173',
 '1952357939',
 '1952360552',
 '1952361954',
 '1952362332',
 '1952364864',
 '1952365680',
 '1952372913',
 '1952377185',
 '1952393071',
 '1952394252',
 '1952406697',
 '1952409311',
 '1952410309',
 '1952412911',
 '1952425472',
 '1952433764',
 '1952447094',
 '1952447776',
 '1952454896',
 '1952460701',
 '1952467951',
 '1952472586',
 '1952472782',
 '1952476779',
 '1952481313',
 '1952488578',
 '1952490880',
 '1952498901',
 '1952500476',
 '1952503920',
 '1952504272',
 '1952512756',
 '1952518980',
 '1952530160',
 '1952537365',
 '1952538065',
 '1952538857',
 '1952541538',
 '1952546449',
 '1

In [13]:
try:
    raw = pd.read_pickle('raw.pickle')
except:
    raw = pd.DataFrame(columns=['IDSubtitleFile','raw_content','size'])
raw.head()

,IDSubtitleFile,raw_content,size
0,123093,"b'1\n00:01:10,000 --> 00:01:13,000\nMARC\nWeek...",69815
1,136803,"b'1\r\n00:00:01,301 --> 00:00:06,261\r\nI dedi...",42263
2,1951858260,"b'1\r\n00:00:09,843 --> 00:00:12,311\r\nClench...",39268
3,1951858261,"b""1\r\n00:00:01,968 --> 00:00:05,631\r\nNo, i\...",62321
4,1951858262,"b'1\r\n00:00:09,843 --> 00:00:12,311\r\nClench...",101861


In [141]:
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
falta_raw = [x[:-4] for x in files if x[:-4] not in list(raw.IDSubtitleFile)]
falta_raw

['1952848105',
 '1952851908',
 '1952857170',
 '1952860750',
 '1952867775',
 '1952871695',
 '1952875395',
 '1952876844',
 '1952880880',
 '1952882441',
 '1952884346',
 '1952888717',
 '1952892890',
 '1952893078',
 '1952893079',
 '1952894055',
 '1952901223',
 '1952902019',
 '1952904722',
 '1952909349',
 '1952911250',
 '1952911369',
 '1952911409',
 '1952911864',
 '1952915927',
 '1952926279',
 '1952934786',
 '1952936935',
 '1952937619',
 '1952939837',
 '1952944734',
 '1952946467',
 '1952946842',
 '1952948645',
 '1952949347',
 '1952953684',
 '1952955401',
 '1952957091',
 '1952960835',
 '1952961629',
 '1952965225',
 '1952966960',
 '1952977170',
 '1952980487',
 '1952983949',
 '1952985833',
 '1952986378',
 '1952987820',
 '1952988742',
 '1952992880',
 '1952994362',
 '1952994446',
 '1952994906',
 '1953001327',
 '1953003649',
 '1953003650',
 '1953004332',
 '1953006014',
 '1953008403',
 '1953008591',
 '1953016305',
 '1953017982',
 '1953018824',
 '1953018842',
 '1953019018',
 '1953019087',
 '19530265

In [142]:
mypath='subs/'
try:
    raw = pd.read_pickle('raw.pickle')
except:
    raw = pd.DataFrame(columns=['IDSubtitleFile','raw_content','size'])

files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
falta_raw = [x[:-4] for x in files if x[:-4] not in list(raw.IDSubtitleFile)]
print(len(falta_raw))
print(raw.shape)

for x in falta_raw:
    with open(mypath + x+'.txt','rb') as f:
        z = str(f.read())
        #print(z)
        c = dict({'IDSubtitleFile': x, 'raw_content' : z, 'size': len(z)})
        raw = raw.append(c,ignore_index=True)
raw.to_pickle('raw.pickle')

files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
falta_raw = [x[:-4] for x in files if x[:-4] not in list(raw.IDSubtitleFile)]
print(len(falta_raw))
print(raw.shape)

800
(19746, 3)
0
(20546, 3)


In [123]:
raw.head()

,IDSubtitleFile,raw_content,size
0,123093,"b'1\n00:01:10,000 --> 00:01:13,000\nMARC\nWeek...",69815
1,136803,"b'1\r\n00:00:01,301 --> 00:00:06,261\r\nI dedi...",42263
2,1951858260,"b'1\r\n00:00:09,843 --> 00:00:12,311\r\nClench...",39268
3,1951858261,"b""1\r\n00:00:01,968 --> 00:00:05,631\r\nNo, i\...",62321
4,1951858262,"b'1\r\n00:00:09,843 --> 00:00:12,311\r\nClench...",101861


In [3]:
mypath='subs/'
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
falta_raw = [x[:-4] for x in files if x[:-4] not in list(raw.IDSubtitleFile)]



NameError: name 'raw' is not defined

In [271]:
import re

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

def cleaner(string):
    whitelist = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ '
    string = string.replace('\n', ' ')
    string = cleanhtml(string)
    string = ''.join(filter(whitelist.__contains__, string.upper())).strip()
    if len(string)>0:
        return string
    else:
        return ''
def get_clean(x):
    with open(mypath + x+'.txt',encoding='utf-8', errors='ignore') as f:
        z = f.read().split('\n\n')
        k = ' '.join([cleaner(zz.split('\n',2)[2]) for zz in z])
        return ' '.join([x for x in list(set(k.split())) if len(x.strip())>3])


In [272]:
for x in falta_raw[:2]:
    c = get_clean(x)
    print(c)

ACTUALLY PIECE RUIN HARM THROWING OTHER SOMETHING DANGXIN BELIEVE GROWING SUPPORT HALT DAUGHTERINLAW ANYWHERE GENTLEMEN SAFELY THEYRE SUNNYII ATTITUDE ARRIVED FULLY NOTES HUTCHES MATCHED LIVES BROUGHT TROUBLED TELL SAID FALLS CLOSE HEAD OPPONENT FOLLOW SEED RELEASING RIFLES BASTARD EVERYTHING QUICKLY MARTIAL RICH LOOKING HEARTS WORLD NEVER LEGACY BECAUSE YOURS SONGFU DIVIDING BOTTLE POLITE EXPERIENCES STAY HEART DEALS GENERAL PICK RABBIT CAREFULLY EXTEND RIDICULOUS WATCHED SONG HANG BROTHER ACTIONS SISTER HOME WING ALIVE SETTLE ALREADY DIRECTLY INSIDE SHOULD HOUJIN REMINISCING CONTINUES PROVINCE SNATCH AMIDA LITTLE MEET FORCE RELAX COVERED AGAIN USEFUL DARE SAVE PERSONS RESIST MASKED PITCH OPEN WHATS DOWN SAME TERRIBLY FAMILY NEXT MADAM INJURED THEN LIKE UNCLE DELAY SOME DIZZY GONE PRACTICE ENEMY SUFFERED ARIDIVX BETROTHED BODY OUTSIDE ANXIOUS WAIT SITUATION ARRIVES POWDER WAITING THROUGH CONQUERED MORNING PEACE RETURNED WHICH DEEP DANGEROUS BEAUTIFUL FAMILIES MANKIND WUDAO SINGLE SUPP

In [296]:
with open(mypath +'1952893019.txt', 'r') as f:
    z = f.read()
    print(z)

In [2]:
mypath='subs/'
try:
    df_clean = pd.read_pickle('df_clean.pickle')
except:
    df_clean = pd.DataFrame(columns=['id','content'])

files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
falta_raw = [x[:-4] for x in files if x[:-4] not in list(df_clean.id)]

for idc in falta_raw:
    #print(idc)
    try:
        c = dict({'id': idc, 'content' : get_clean(idc)})
        df_clean = df_clean.append(c,ignore_index=True)
    except:
        print(idc)
df_clean.to_pickle('df_clean.pickle')

1952357939
1952538857
1952613861
1952668884
1952739540
1952780447
1952819932
1952826429
1952827081
1952837415
1952839835
1952840399
1952840842
1952844189
1952844194
1952846517
1952848274
1952854225
1952855618
1952858593
1952860125
1952862379
1952863405
1952863431
1952864803
1952864953
1952865655
1952868950
1952869409
1952869761
1952873994
1952874176
1952874589
1952876566
1952876927
1952878170
1952878714
1952879820
1952881408
1952882603
1952882811
1952886105
1952887546
1952889038
1952889866
1952890630
1952890750
1952892219
1952892907
1952893019
1952893485
1952896718
1952897303
1952897338
1952897888
1952897900
1952899535
1952899727
1952900059
1952902096
1952902340
1952902971
1952902994
1952904722
1952905880
1952906603
1952907031
1952907236
1952907653
1952907957
1952909370
1952909708
1952910055
1952910795
1952911479
1952911862
1952911966
1952912100
1952912241
1952912272
1952912431
1952912492
1952912839
1952912952
1952913062
1952913069
1952913149
1952914016
1952914974
1952915978
1952916395

1955402817
1955402871
1955403288
1955403340
1955403478
1955407050
1955407440
1955407448
1955407781
1955408976
1955409496
1955409688
1955409699
1955409960
1955410311
1955410406
1955410492
1955411362
1955411380
1955411973
1955412013
1955413371
1955413431
1955414338
1955414713
1955415054
1955415473
1955417294
1955417497
1955418447
1955419710
1955419835
1955420180
1955420378
1955420394
1955420720
1955421592
1955421747
1955422272
1955422517
1955423651
1955424384
1955424666
1955424927
1955428395
1955428566
1955428753
1955428818
1955429727
1955430215
1955430387
1955431079
1955431536
1955431981
1955432102
1955432453
1955434734
1955435760
1955436212
1955437364
1955437366
1955437771
1955438497
1955439327
1955439430
1955439669
1955440067
1955441817
1955442370
1955442589
1955443684
1955444145
1955444342
1955445515
1955447237
1955447736
1955448226
1955449662
1955449908
1955449912
1955450978
1955451978
1955452395
1955453079
1955454633
1955454950
1955455413
1955456114
1955456183
1955458196
1955458197

In [3]:
all_words = list(' '.join(list(df_clean.content)).split())


In [4]:
all_words_dict = list(set(all_words))

In [99]:
print(len(all_words))
print(len(all_words_dict))

25690366
454293


In [100]:
#all_words = all_words[:1000]
#all_words_dict = list(set(all_words))

In [101]:
all_words.count(all_words_dict[0])

1

In [102]:
words = pd.DataFrame(columns=['word','count_word'])

In [103]:
words.word = all_words


In [104]:
words['count_word'] = 1
words

,word,count_word
0,ACTUALLY,1
1,PIECE,1
2,RUIN,1
3,HARM,1
4,THROWING,1
5,OTHER,1
6,SOMETHING,1
7,DANGXIN,1
8,BELIEVE,1
9,GROWING,1


In [105]:
words2 = words.groupby(['word']).sum()

In [106]:
words2.shape

(454293, 1)

In [107]:
words2 = words2.sort_values('count_word', ascending=False)

In [14]:
for i in range(20):
    print(i,words2[words2.count_word > i].shape[0])

0 454293
1 217508
2 156330
3 128751
4 111939
5 100630
6 92293
7 85409
8 79727
9 74926
10 70485
11 66728
12 63429
13 60664
14 58190
15 55983
16 53835
17 51991
18 50391
19 48875


In [15]:
for i in range(1000):
    print(i,words2.index[i],words2.count_word[i])

0 THIS 20583
1 HAVE 20548
2 WHAT 20525
3 THAT 20522
4 HERE 20521
5 WITH 20499
6 YOUR 20487
7 KNOW 20480
8 FROM 20466
9 THERE 20428
10 WERE 20413
11 COME 20408
12 LIKE 20401
13 THEY 20392
14 JUST 20369
15 ABOUT 20357
16 BACK 20356
17 TAKE 20338
18 WHERE 20334
19 WILL 20329
20 GOOD 20326
21 TIME 20326
22 WHEN 20319
23 WANT 20310
24 THEM 20287
25 THEN 20270
26 THINK 20257
27 RIGHT 20255
28 GOING 20249
29 LOOK 20222
30 MORE 20216
31 WELL 20197
32 NEED 20170
33 THATS 20168
34 BEEN 20166
35 SOME 20105
36 DOWN 20096
37 NEVER 20094
38 MAKE 20082
39 OVER 20068
40 SOMETHING 20067
41 DONT 20043
42 WOULD 20028
43 GIVE 20024
44 YOURE 20016
45 ONLY 20012
46 TELL 20011
47 MUCH 19977
48 COULD 19939
49 SHOULD 19888
50 AGAIN 19886
51 SAID 19872
52 DOING 19861
53 WHATS 19801
54 EVEN 19782
55 NOTHING 19780
56 AWAY 19769
57 STILL 19756
58 FIRST 19722
59 CANT 19706
60 BEFORE 19693
61 ANYTHING 19674
62 FIND 19672
63 STOP 19666
64 HELP 19654
65 SURE 19620
66 REALLY 19612
67 PEOPLE 19605
68 THESE 19603
69 BECA

762 CONGRATULATIONS 6664
763 HONEST 6662
764 BEGINNING 6654
765 NINE 6653
766 DATE 6650
767 PAPER 6650
768 BUNCH 6647
769 EATING 6644
770 ESPECIALLY 6642
771 LAUGH 6639
772 REMOVE 6639
773 BRAND 6615
774 FORGIVE 6601
775 FORCE 6592
776 SOUL 6591
777 BECAME 6590
778 INTERESTED 6582
779 ANGRY 6561
780 WAYS 6555
781 DADDY 6549
782 BEGIN 6536
783 PUSH 6514
784 WHETHER 6493
785 DRIVING 6491
786 RECORD 6476
787 STATION 6443
788 OBVIOUSLY 6437
789 STAYING 6435
790 TURNS 6421
791 FOLLOWING 6419
792 FOOT 6397
793 ACCEPT 6396
794 TRIP 6392
795 THEMSELVES 6360
796 ALLOWED 6359
797 TWICE 6354
798 JOKE 6345
799 CERTAIN 6342
800 PRIVATE 6338
801 ASSHOLE 6322
802 BROTHERS 6313
803 PRODUCT 6308
804 ENGLISH 6308
805 OFTEN 6306
806 DECIDE 6302
807 FAVORITE 6287
808 WASTE 6274
809 LOVELY 6271
810 GROUP 6264
811 JUMP 6263
812 ROCK 6258
813 RULES 6255
814 CLOSER 6246
815 SMILE 6234
816 QUIT 6227
817 USUALLY 6224
818 DRUNK 6214
819 CELL 6213
820 FURTHER 6211
821 SPOT 6203
822 OTHERWISE 6201
823 LEARNED 6194

In [37]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
  
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer 

ps = PorterStemmer() 

def tokenize_string(s):
    word_tokens = word_tokenize(s) 
    stop_words = set(stopwords.words('english')) 
    #filtered_sentence = list(set([ps.stem(w)+':'+w for w in word_tokens if not w in stop_words] ))
    filtered_sentence = list(set([ps.stem(w) for w in word_tokens if not w in stop_words] ))
    filtered_sentence.sort()
    
    return ' '.join(filtered_sentence)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jujubinha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jujubinha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [46]:
words2 = words2.reset_index()
#words2['stem'] = words2['word'].apply(tokenize_string)
words2['stem'] = words2['word'].apply(ps.stem)

In [47]:
words2.sort_values('stem')

,index,word,count_word,stem
253947,253947,AING,1,a
101661,101661,AAAS,5,aaa
46121,46121,AAAA,22,aaaa
145488,145488,AAAAA,3,aaaaa
125746,125746,AAAAAA,4,aaaaaa
210843,210843,AAAAAAA,2,aaaaaaa
220066,220066,AAAAAAAAAAAAA,1,aaaaaaaaaaaaa
221856,221856,AAAAAAAAAAAAAAAAQEE,1,aaaaaaaaaaaaaaaaqe
213162,213162,AAAAAAAAAAAAARGH,2,aaaaaaaaaaaaargh
152587,152587,AAAAAAAAAAAAH,3,aaaaaaaaaaaah


In [50]:
words3 = words2.groupby('stem').sum()

In [53]:
words3.sort_values('count_word', ascending = False)

,index,count_word
stem,,
look,286178,64571
want,71181,56799
happen,1662165,55631
work,264974,54793
come,833927,54435
your,353,54259
call,159743,53801
live,495808,50015
make,464791,49400


In [61]:
words3 = words3.reset_index()
for i in range(200):
    print(i,words3[words3.count_word > i].shape[0])

0 378474
1 173399
2 119248
3 96251
4 81902
5 72918
6 66010
7 60730
8 56231
9 52639
10 49274
11 46416
12 43885
13 41882
14 40034
15 38513
16 36912
17 35600
18 34413
19 33357
20 32359
21 31507
22 30719
23 29974
24 29279
25 28697
26 28103
27 27551
28 26989
29 26485
30 26017
31 25580
32 25152
33 24755
34 24386
35 24036
36 23683
37 23351
38 23054
39 22755
40 22436
41 22122
42 21836
43 21600
44 21323
45 21063
46 20812
47 20595
48 20368
49 20141
50 19939
51 19737
52 19547
53 19351
54 19171
55 18988
56 18823
57 18655
58 18471
59 18309
60 18142
61 17967
62 17813
63 17652
64 17518
65 17372
66 17227
67 17081
68 16956
69 16808
70 16683
71 16560
72 16429
73 16315
74 16202
75 16083
76 15974
77 15851
78 15768
79 15659
80 15559
81 15450
82 15348
83 15245
84 15133
85 15025
86 14910
87 14800
88 14712
89 14637
90 14549
91 14466
92 14380
93 14289
94 14190
95 14104
96 14017
97 13933
98 13860
99 13799
100 13726
101 13624
102 13549
103 13478
104 13411
105 13350
106 13275
107 13202
108 13126
109 13047
110 129

In [66]:
words3 = words3[['stem','count_word']].sort_values('count_word', ascending=False)
words3

,stem,count_word
193464,look,64571
357829,want,56799
141913,happen,55631
366638,work,54793
62327,come,54435
373769,your,54259
48593,call,53801
191163,live,50015
199132,make,49400
322710,talk,47734


In [67]:
words3.to_pickle('words3.pickle')

In [108]:
words2

,count_word
word,
THIS,20583
HAVE,20548
WHAT,20525
THAT,20522
HERE,20521
WITH,20499
YOUR,20487
KNOW,20480
FROM,20466


In [109]:
words2 = words2.reset_index()
words2

,word,count_word
0,THIS,20583
1,HAVE,20548
2,WHAT,20525
3,THAT,20522
4,HERE,20521
5,WITH,20499
6,YOUR,20487
7,KNOW,20480
8,FROM,20466
9,THERE,20428


In [110]:
words2 = words2[['word','count_word']]

w2 = list(words2['word'][:50000])
df2 = df_clean
shuffle(w2)
w2

['NONPLAYER',
 'BACCHANALIA',
 'COMPELS',
 'PUNISHMENT',
 'SYNAPSE',
 'FLICKERED',
 'FAVOR',
 'BULKHEADS',
 'SUBSECTION',
 'GALLOPS',
 'ANTITRUST',
 'DEBRIS',
 'LIFEALTERING',
 'MADEMOISELLE',
 'DEFAULTBEFORE',
 'HIMSEIF',
 'HYPERTORQUE',
 'DOUGIE',
 'SHEEN',
 'REJOICES',
 'WRYLY',
 'IRANS',
 'POTSHOTS',
 'SASHAY',
 'YOUTUBES',
 'CABERNET',
 'MONKS',
 'ESTHERS',
 'LICORICE',
 'FELINE',
 'DORSET',
 'MAGICIANS',
 'DILDO',
 'SURRENDERING',
 'MARRIAGEABLE',
 'CLOSEST',
 'SNEERED',
 'MINDER',
 'HALIFAX',
 'PARTHENON',
 'CROSSLEGGED',
 'TRANSLATES',
 'STONEWATCH',
 'SEETHE',
 'EXACTING',
 'BARROW',
 'UNKIND',
 'DINNER',
 'SLOANES',
 'FIVEPOUND',
 'TASTELESS',
 'OVERLAY',
 'FLOODIN',
 'WHOWHOS',
 'REELED',
 'UHMM',
 'INANIMATE',
 'BERYLLIUM',
 'HANDLERS',
 'FRANCHISES',
 'QUARTET',
 'PAINFULLY',
 'HOMEWORK',
 'BIOFUEL',
 'BREEZEWAY',
 'LAKSHMAN',
 'BREATHER',
 'BRIAN',
 'RESALE',
 'CONGRATULATES',
 'WEAVE',
 'NIECES',
 'OVERSEERS',
 'BELLS',
 'BARFING',
 'ROUND',
 'STEVENS',
 'CASES',
 'HORCR

In [ ]:
k=500
l = int(len(w2)/k)

for v in range(l):
    print(v,l)
    for w in w2[v*k:v*k+k]:
        df2['word_'+str(w)] = df2['content'].str.count(w)
    df2.to_pickle('df2.pickle')
df2

0 100
1 100
2 100
3 100
4 100
5 100
6 100
7 100
8 100
9 100
10 100
11 100
12 100
13 100
14 100
15 100
16 100
17 100
18 100
19 100
20 100
21 100
22 100
23 100


In [20]:
words2.head()


,count_word
word,
THIS,20583
HAVE,20548
WHAT,20525
THAT,20522
HERE,20521


In [21]:
df_clean.head()

,id,content
0,1952749712,ACTUALLY PIECE RUIN HARM THROWING OTHER SOMETH...
1,1952766057,MANISH FLIRT BLAMING FEBRUARY PROFESSION OTHER...
2,1952827068,ONLINE SWEAR MANISH NEWS CONFESSED JUDGEMENT M...
3,1952858703,SITE ONLINE ASSIGNED UNTIL BLOCKS SWEAR UPSTAI...
4,1952862063,UPSTAIRS AIIVE REAIIZED SOMETHING OTHER SIEEPI...


In [185]:
df = df_clean #.head()
df

,id,content
0,1952749712,ACTUALLY PIECE RUIN HARM THROWING OTHER SOMETH...
1,1952766057,MANISH FLIRT BLAMING FEBRUARY PROFESSION OTHER...
2,1952827068,ONLINE SWEAR MANISH NEWS CONFESSED JUDGEMENT M...
3,1952858703,SITE ONLINE ASSIGNED UNTIL BLOCKS SWEAR UPSTAI...
4,1952862063,UPSTAIRS AIIVE REAIIZED SOMETHING OTHER SIEEPI...
5,1952863503,DISTRICT BLAMING OTHER DISAPPEAR BELIEVE INQUI...
6,1952869763,NEWS APPEAR BRESSLER CYANIDE SOMETHING OTHER B...
7,1952869919,NEWS UNTIL APPEAR UPSTAIRS ALTHOUGH SEAMLESSLY...
8,1952870004,NEWS UNTIL APPEAR UPSTAIRS SEAMLESSLY BRESSLER...
9,1952870009,NEWS UNTIL APPEAR UPSTAIRS ALTHOUGH SEAMLESSLY...


In [ ]:
dfx = pd.DataFrame([])
total = df.shape[0]
quantum = int(total/250)+1
print('total',total,'quantum',quantum)
for j in range(0,250):
    kk=0
    e = dict()
    for line in df[j*quantum:j*quantum+quantum].iterrows():
        kk = kk + 1
        print('.',end='',flush=True)
        content = line[1][1]
        idc = line[1][0]
        e[kk] = dict({'id':idc})
        for w in content.split():
            if w in w2:
                e[kk]['word_'+w] = True
    for f in e:
        dfx = dfx.append(e[f], ignore_index=True)
    print('\n',j,dfx.shape)
    dfx.to_pickle('df2.pickle')

dfx

total 20694 quantum 83
...................................................................................
 0 (83, 16383)
...................................................................................
 1 (166, 22771)
...................................................................................
 2 (249, 27014)
...................................................................................
 3 (332, 30583)
...................................................................................
 4 (415, 33274)
...................................................................................
 5 (498, 35718)
...................................................................................
 6 (581, 37622)
...................................................................................
 7 (664, 39472)
...................................................................................
 8 (747, 40921)
..............................................................................

In [ ]:

df

In [88]:
w3 = list(words3[:50000]['stem'])

In [83]:
df = df_clean
df['stem'] = df['content'].apply(tokenize_string)
df

,id,content,word_piece,stem
0,1952749712,ACTUALLY PIECE RUIN HARM THROWING OTHER SOMETH...,1,abbot abbott abl about achiev action actual ad...
1,1952766057,MANISH FLIRT BLAMING FEBRUARY PROFESSION OTHER...,2,NA ablaz about abroad absolut accept accid acc...
2,1952827068,ONLINE SWEAR MANISH NEWS CONFESSED JUDGEMENT M...,2,NA ablaz about abroad absolut accept accord ac...
3,1952858703,SITE ONLINE ASSIGNED UNTIL BLOCKS SWEAR UPSTAI...,0,NA TA abort about abov accid accompli account ...
4,1952862063,UPSTAIRS AIIVE REAIIZED SOMETHING OTHER SIEEPI...,0,NA about accept accid accord after again ahead...
5,1952863503,DISTRICT BLAMING OTHER DISAPPEAR BELIEVE INQUI...,1,abandon abl about accept accomplic accord acco...
6,1952869763,NEWS APPEAR BRESSLER CYANIDE SOMETHING OTHER B...,0,NA abl about access accessori afraid africa af...
7,1952869919,NEWS UNTIL APPEAR UPSTAIRS ALTHOUGH SEAMLESSLY...,1,NA abl about absenc absolut access accessori a...
8,1952870004,NEWS UNTIL APPEAR UPSTAIRS SEAMLESSLY BRESSLER...,1,NA abl about absenc absolut access accessori a...
9,1952870009,NEWS UNTIL APPEAR UPSTAIRS ALTHOUGH SEAMLESSLY...,1,NA abl about absenc absolut access accessori a...


In [92]:
shuffle(w3)

for w in w3[:100]:
    df['word_'+str(w)] = df['stem'].str.count(w)
df

,id,content,word_piece,stem,word_look,word_want,word_happen,word_work,word_come,word_wherendid,word_cergi,word_bubblesfsorgfrzchchffffffbordbposshadar,word_cocoain,word_httpspastebincomxbfrtxj,word_polakkan,word_knowthatwel,word_crimefre,word_thingammi,word_hersni,word_rerend,word_belgrad,word_sscscriptspecialmovemodo,word_exceptgunnar,word_transcast,word_embankmentndown,word_bynow,word_viewfind,word_ungass,word_mikag,word_konnerreuth,word_thenhomestead,word_defaultxiaoyu,word_troopsfrom,word_vijam,word_trumeau,word_vikingsmight,word_sonequinha,word_sivachidambaram,word_fishfac,word_singlepar,word_lneat,word_hiha,word_jennyfuckingrifkin,word_guilingj,word_defaultshimura,word_kusadasi,word_mutafa,word_ecosustain,word_ratiocin,word_lifeold,word_abitch,word_nightcrawt,word_aanvoerd,word_sanctuari,word_smooz,word_grandmath,word_wordsnoh,word_rafe,word_benfica,word_defusenthi,word_handstitch,word_porazi,word_highcodewhat,word_nolet,word_chahta,word_locomobil,word_xico,word_reord,word_aypmebbqnazrtrapruab,word_ventaja,word_aaqabqqeuhpratqptttq,word_sidemen,word_henn,word_nadao,word_shahida,word_meaningnto,word_sleighwhat,word_myilythink,word_anotherfast,word_mba,word_jeongjong,word_dadget,word_coursenyouv,word_eurasia,word_stickyickyicki,word_woroniecki,word_coigni,word_renchang,word_beennpart,word_decorat,word_thew,word_fuchuckl,word_criterianno,word_hofbrau,word_dyingisnt,word_ambika,word_surpren,word_tucumcari,word_unbutton,word_moko,word_upget,word_catesbi,word_shipwreck,word_hovv,word_paramount,word_adam,word_topsail,word_midlevel,word_uncoop,word_catcal,word_markeddefaultntpeffectmr,word_joli,word_landsberg,word_badass,word_ulyss,word_curabl,word_spag,word_kojima,word_junctur,word_polonium,word_newtonjohn,word_whoopass,word_alfheim,word_smetim,word_loan,word_manic,word_mortim,word_hextal,word_iend,word_steal,word_sassonack,word_teleplay,word_chingada,word_marshi,word_wallop,word_cartwright,word_lara,word_tenden,word_mege,word_blanki,word_lasset,word_doublea,word_analyz,word_markeddefaultntpeffectif,word_everpres,word_synchron,word_yucca,word_asymmetr,word_parvati,word_heya,word_reestablish,word_girltalk,word_mcmissil,word_misdial,word_polygamist,word_pedersen,word_somebodyl,word_neza,word_indomit,word_emanuel,word_fang,word_beez,word_sontag,word_prolif,word_greatest,word_carleton,word_heatstrok,word_willem,word_sherbert,word_connal,word_cyr,word_shake,word_evemh,word_riffraff,word_rhino,word_lidic,word_rigor,word_zucker,word_bukowski,word_smucksdinnin,word_shei,word_nothingi,word_woolli,word_jedediah,word_robbi,word_halfdozen,word_anarch,word_parlour,word_herbert,word_mission,word_wisdom,word_memori,word_budweis,word_gyeong,word_blob,word_finklestein,word_fifthyear,word_keenan,word_squiggl,word_rubinstein,word_bahia,word_knuckl,word_shard,word_abroad,word_deltoid,word_midtwenti,word_licker,word_expertis,word_loofah,word_woosh,word_cinematographi,word_chuckl,word_variou,word_markeddefaultntpeffector,word_cowan,word_burbag,word_herna,word_ipod,word_thenth,word_infield,word_carey,word_shazam,word_marchal,word_dasher,word_dunk,word_unstick,word_pbbt,word_yekaterina,word_epistol,word_wealthier,word_oohoohooh,word_twentyfirst,word_tammi,word_milori,word_misa,word_cometh,word_carson,word_cla,word_fernandez,word_remo,word_sicilian,word_surfacetosea,word_agonis,word_restaur,word_sexykitten,word_finetooth,word_atol,word_euphoria,word_tertiari,word_matriarch,...,word_iceland,word_youncan,word_pear,word_albani,word_rosemoor,word_defaultsearch,word_kolfax,word_salzburg,word_beachfront,word_ineffect,word_storekeep,word_hesketh,word_aramaki,word_tweak,word_anytim,word_atlant,word_unschedul,word_godpar,word_ummthre,word_donor,word_shetti,word_truant,word_mediaplex,word_welljust,word_mata,word_restitut,word_libelist,word_bruci,word_injun,word_yoram,word_reasn,word_facetofac,word_lucienn,word_bottomless,word_guess,word_highstrung,word_overhang,word_airstream,word_annemari,word_conceptu,word_yeti,word_ramzan,word_heisman,word_riddanc,word_advert